In [ ]:
# | default_exp _components.aiokafka_producer_manager

In [ ]:
# | export

import asyncio
from contextlib import asynccontextmanager, contextmanager
from typing import *

import anyio
from aiokafka import AIOKafkaProducer

from fastkafka._components.logger import get_logger

In [ ]:
import unittest.mock
from os import environ

from fastkafka._components.logger import supress_timestamps
from fastkafka.testing import (
    create_and_fill_testing_topic,
    nb_safe_seed,
    true_after,
)

In [ ]:
seed = nb_safe_seed("_components.aiokafka_producer_loop")

In [ ]:
# | notest
# allows async calls in notebooks

import nest_asyncio

nest_asyncio.apply()

In [ ]:
# | export

logger = get_logger(__name__)

In [ ]:
supress_timestamps()
logger = get_logger(__name__, level=1)
logger.info("ok")

[INFO] __main__: ok


In [ ]:
kafka_server_url = environ["KAFKA_HOSTNAME"]
kafka_server_port = environ["KAFKA_PORT"]

kafka_config = {"bootstrap.servers": f"{kafka_server_url}:{kafka_server_port}"}

In [ ]:
# | export


@asynccontextmanager
async def _aiokafka_producer_manager(  # type: ignore
    producer: AIOKafkaProducer, *, max_buffer_size: int = 10_000
):
    """Write docs

    Todo: add batch size if needed
    """

    logger.info("_aiokafka_producer_manager(): Starting...")

    async def send_message(receive_stream):
        async with receive_stream:
            async for topic, msg in receive_stream:
                fut = await producer.send(topic, msg)
                msg = await fut

    send_stream, receive_stream = anyio.create_memory_object_stream(
        max_buffer_size=max_buffer_size
    )

    logger.info("_aiokafka_producer_manager(): Starting task group")
    async with anyio.create_task_group() as task_group:
        logger.info("_aiokafka_producer_manager(): Starting send_stream")
        task_group.start_soon(send_message, receive_stream)
        async with send_stream:
            yield send_stream
            logger.info("_aiokafka_producer_manager(): Exiting send_stream")
        logger.info("_aiokafka_producer_manager(): Exiting task group")
    logger.info("_aiokafka_producer_manager(): Finished.")

In [ ]:
@contextmanager
def mock_AIOKafkaProducer_send():
    with unittest.mock.patch("__main__.AIOKafkaProducer.send") as mock:

        async def _f():
            pass

        mock.return_value = asyncio.create_task(_f())

        yield mock

In [ ]:
num_msgs = 15
topic = "topic"
msg = b"msg"
msgs = [(topic, msg) for _ in range(num_msgs)]
calls = [unittest.mock.call(topic, msg) for _ in range(num_msgs)]

with mock_AIOKafkaProducer_send() as send_mock:
    producer = AIOKafkaProducer()
    async with _aiokafka_producer_manager(producer) as send_stream:

        for msg in msgs:
            send_stream.send_nowait(msg)

        await producer.stop()
    #     await producer_loop_generator.__aexit__(None, None, None)

    send_mock.assert_has_calls(calls)

[INFO] __main__: _aiokafka_producer_manager(): Starting...
[INFO] __main__: _aiokafka_producer_manager(): Starting task group
[INFO] __main__: _aiokafka_producer_manager(): Starting send_stream
[DEBUG] aiokafka.producer.producer: The Kafka producer has closed.
[INFO] __main__: _aiokafka_producer_manager(): Exiting send_stream
[INFO] __main__: _aiokafka_producer_manager(): Exiting task group
[INFO] __main__: _aiokafka_producer_manager(): Finished.


In [ ]:
# | export


class AIOKafkaProducerManager:
    def __init__(self, producer: AIOKafkaProducer, *, max_buffer_size: int = 1_000):  # type: ignore
        self.producer = producer
        self.max_buffer_size = max_buffer_size

    async def start(self) -> None:
        logger.info("AIOKafkaProducerManager.start(): Entering...")
        await self.producer.start()
        self.producer_manager_generator = _aiokafka_producer_manager(self.producer)
        self.send_stream = await self.producer_manager_generator.__aenter__()
        logger.info("AIOKafkaProducerManager.start(): Finished.")

    async def stop(self) -> None:
        # todo: try to flush messages before you exit
        logger.info("AIOKafkaProducerManager.stop(): Entering...")
        await self.producer_manager_generator.__aexit__(None, None, None)
        logger.info("AIOKafkaProducerManager.stop(): Stoping producer...")
        await self.producer.stop()
        logger.info("AIOKafkaProducerManager.stop(): Finished")

    def send(self, topic: str, msg: bytes) -> None:
        self.send_stream.send_nowait((topic, msg))

In [ ]:
producer = AIOKafkaProducer(bootstrap_servers=kafka_config["bootstrap.servers"])
manager = AIOKafkaProducerManager(producer)
await manager.start()
manager.send("topic", b"msg")
await manager.stop()
logger.info("Stopped")

[INFO] __main__: AIOKafkaProducerManager.start(): Entering...
[DEBUG] aiokafka.producer.producer: Starting the Kafka producer
[DEBUG] aiokafka: Attempting to bootstrap via node at tvrtko-fast-kafka-api-kafka-1:9092
[DEBUG] aiokafka.conn: <AIOKafkaConnection host=tvrtko-fast-kafka-api-kafka-1 port=9092> Request 1: MetadataRequest_v0(topics=[])
[DEBUG] aiokafka.conn: <AIOKafkaConnection host=tvrtko-fast-kafka-api-kafka-1 port=9092> Response 1: MetadataResponse_v0(brokers=[(node_id=1001, host='75d5a1be66b3', port=9092), (node_id=1003, host='40c27daf393d', port=9092), (node_id=1002, host='681f4568022c', port=9092)], topics=[(error_code=0, topic='my_topic_1', partitions=[(error_code=0, partition=0, leader=1003, replicas=[1003], isr=[1003])]), (error_code=0, topic='my_test_topic_2', partitions=[(error_code=0, partition=0, leader=1002, replicas=[1002], isr=[1002])]), (error_code=0, topic='training_status', partitions=[(error_code=0, partition=0, leader=1001, replicas=[1001, 1002, 1003], isr=[

[DEBUG] aiokafka.conn: Closing connection at 40c27daf393d:9092
[DEBUG] aiokafka.producer.producer: Kafka producer started
[INFO] __main__: _aiokafka_producer_manager(): Starting...
[INFO] __main__: _aiokafka_producer_manager(): Starting task group
[INFO] __main__: _aiokafka_producer_manager(): Starting send_stream
[INFO] __main__: AIOKafkaProducerManager.start(): Finished.
[INFO] __main__: AIOKafkaProducerManager.stop(): Entering...
[INFO] __main__: _aiokafka_producer_manager(): Exiting send_stream
[INFO] __main__: _aiokafka_producer_manager(): Exiting task group
[DEBUG] aiokafka.producer.producer: Sending (key=None value=b'msg') to TopicPartition(topic='topic', partition=0)
[DEBUG] aiokafka: Initiating connection to node 1001 at 75d5a1be66b3:9092
[DEBUG] aiokafka.conn: <AIOKafkaConnection host=75d5a1be66b3 port=9092> Request 1: ApiVersionRequest_v0()
[DEBUG] aiokafka.conn: <AIOKafkaConnection host=75d5a1be66b3 port=9092> Response 1: ApiVersionResponse_v0(error_code=0, api_versions=[(a